# PA193 Seminar - RNGs
This notebook contains code for several tasks treated in this seminar. 

# Task 1: determinism of PRNG

We will work with PRNG implemented in [random](https://docs.python.org/3/library/random.html) package. See first 4 methods (`seed, setstate, getstate, randbytes`) in the documentation. 
 1. Import **random** package.
 2. Generate (and print) 3 random bytes.  
 3. Print out bytes in hexadecimal form (use `.hex()` method of bytes). Execute cell 2x - use Run or Ctrl+Enter.  
 4. Save state (int `state` variable) of the PRNG right after the seeding (before generation of bytes).  
 5. Set the state of PRNG and generate the same result as in step 4. 

In [137]:
import random 
rnd_bytes = random.randbytes(10)
print(rnd_bytes)
print(rnd_bytes.hex())

random.seed(2)
state = random.getstate()
print(random.randbytes(10).hex())

# print(state)
random.setstate(state)
print(random.randbytes(10).hex())

b'\xc8\xaf[\xd9\x9f&z\x0er\x17'
c8af5bd99f267a0e7217
73a9bef499bbf4dca4f2
73a9bef499bbf4dca4f2


# LCG
Standard PRNG functions are very fast but also very insecure. 
 * In python, PRNG [implemented](https://svn.python.org/projects/python/branches/release32-maint/Lib/random.py) in random module is [Mersenne Twister](https://en.wikipedia.org/wiki/Mersenne_Twister) with state formed by 625 32-bit integers. 
 * In other languages (C, Java, Rust) LCG is typically used. Internal state of LCG is **single** value (state) updated iterativelly as $$state = (state*a+c) \pmod m.$$ Overview of constants `a,c,m` used by the LCG for several languages can be found [LCG params and generators](https://en.wikipedia.org/wiki/Linear_congruential_generator).  
 <span style="color:red">In LCG, state (new or old) is typically returned as generated random value!!</span>

# Task 2: Forward/backward predictability  
 0. Check that the parameters of LCG generators (glibc, ZX81) corresponds to values published here [LCG params and generators](https://en.wikipedia.org/wiki/Linear_congruential_generator) 
 1. Attacker knowns that the **glibc** generated number 1406932606. Why he is able to find the internal state of the **RNG**? What is the problem? 
 2. Use appropriate seed and generate next 9 values until. 
 3. Are you able to create "inverse" LCG that goes in opposite directions? Start from the last value generated in 2. end with the first.  
  - **HINT**: $x_{i+1} = a*x_{i}+c \pmod m \implies x_{i} = a^{-1}*x_{i+1}-(a^{-1}*c) \pmod m$
 

In [5]:
class PRNG:
    def __init__(self, a, c, m):
        self.a, self.c, self.m = a, c, m
        self.srand(0)

    def srand(self, seed):
        self.state = seed

    def rand(self):
        self.state = (self.state * self.a + self.c) % self.m
        return self.state
    
ZX81 = PRNG(a=75, c=74, m=2**16 + 1)
rngs = [ZX81.rand() for _ in range(10) ]

glibc = PRNG(a=1103515245, c=12345, m=2**31)

rngs = [glibc.rand() for _ in range(10) ]
print(rngs)

a_inverse = pow(1103515245,-1,2**31)
glibc_backward = PRNG(a=a_inverse, c=-12345*a_inverse, m=2**31)
glibc_backward.srand(551188310)

rngs = [glibc_backward.rand() for _ in range(10) ]
print(rngs)

[12345, 1406932606, 654583775, 1449466924, 229283573, 1109335178, 1051550459, 1293799192, 794471793, 551188310]
[794471793, 1293799192, 1051550459, 1109335178, 229283573, 1449466924, 654583775, 1406932606, 12345, 0]


# Task 3: RNG sequence  
 1. Use ZX81 seeded with 0 and generate 130000 values - assign `sequence` to this list of values.  
 2. Check that generated sequence is periodic with period 65536 (i.e. there are 65536 different values, and `sequence[0]==sequence[65535]`). 
 3. Verify that the only value missing in the sequence is 65536. 
 4. Since sequence consists of almost all values there is one large cycle. I.e. different seed just defines different start within the cycle = shifted sequence.  
 5. Verify that other sequence is 65536, 65536, ... .  

In [8]:
ZX81.srand(0)
sequence = [ZX81.rand() for i in range(65536)]
print(65536 in sequence)
print(len(set(sequence)))

ZX81.srand(65536)
sequence = [ZX81.rand() for i in range(10)]
print(sequence)

False
65536
[65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536]


# Task 4: CSPRNG(but!!!)
Hash functions `SHA1` can be used as one-way functions (hard to invert) to create CSPRNG. One-wayness of SHA1 doesn't allow to compute previous state of RNG from the generated values. 
 1. Fix the generation process (rand function does not update the internal state). 
 2. Verify that SHA1 used same way (as state update func. and output func.) leaks internal state of RNG. 
 3. Fix given security issue e.g. functions used should be different - it suffices to use SHA1(x), SHA(x^2). On the other hand   SHA(x), SHA1(x)^2 is still problematic! 
 - ^ represents bit flip. 

In [29]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

def SHA1(message: bytes):
    digest = hashes.Hash(hashes.SHA1())
    digest.update(message)
    return digest.finalize() 


class CSPRNG_hash:
    def __init__(self, state_size = 1):
        self.size = state_size
        seed = bytes.fromhex("00"*state_size)
        self.srand(seed)

    def srand(self, seed: bytes):
        self.state = seed
        
    def RNG_state_update(self):
        self.state = SHA1(self.state)[:self.size]    
    
    def rand(self):
        self.RNG_state_update()
        self.state = bytearray(self.state)
        self.state[-1] ^= 2
        return SHA1(self.state)[:self.size]
    
csprng_hash = CSPRNG_hash()

seed = bytes.fromhex("12")
csprng_hash.srand(seed)
sequence = [csprng_hash.rand() for i in range(10)]
print(sequence)

[b'\x8b', b'<', b'\t', b'\x06', b'\xa4', b'M', b'\x08', b'\xad', b'\xe2', b'\xc2']


# Task 5: CSPRNG(but - small cycles)
 1. Use `csprng` generator and generate sequence of 30 values. Using `find_cycle` function find periodic and pre-periodic (p-periodic) part of the sequence. 
 2. Verify (try few different seeds) that periods are affected by birthday paradox i.e. period is roughly sqrt(N) instead of N.

In [30]:
def find_cycle(sequence):
    for value in sequence:
        if sequence.count(value) >= 2:
            idx_first = sequence.index(value)
            idx_second = sequence[idx_first+1:].index(value)
            return sequence[idx_first: idx_first + 1 + idx_second]
    return None
seed = bytes.fromhex("11")
csprng_hash.srand(seed)
sequence  = [csprng_hash.rand() for _ in range(30) ]
print(sequence)
cycle = find_cycle(sequence)
print(cycle)

[b'R', b'Q', b'\x02', b'[', b'#', b'\n', b'\x8d', b'd', b'J', b'|', b'\xfb', b'\xa1', b'\x12', b'n', b'\x07', b'\x8d', b'd', b'J', b'|', b'\xfb', b'\xa1', b'\x12', b'n', b'\x07', b'\x8d', b'd', b'J', b'|', b'\xfb', b'\xa1']
[b'\x8d', b'd', b'J', b'|', b'\xfb', b'\xa1', b'\x12', b'n', b'\x07']


# Task 6: CSPRNG with CRT mode

 1. The following CSPRNG `csprng` is secure with maximal period. But it uses always the same key! The only way how to recover from the state compromise is reseeding the generator with new counter value. Change the implementation so the AES key could be also reseeded.

In [148]:
class CSPRNG_CRT:
    def __init__(self):
        seed = bytes.fromhex("00"*16)
        self.srand(seed)
        
    def srand(self, seed: bytes): 
        key = bytes.fromhex("00"*16)
        cipher = Cipher(algorithms.AES(key), modes.CTR(seed))
        self.encryptor = cipher.encryptor() 

    
    def rand(self):
        msg =  bytes.fromhex("00"*16)
        ct = self.encryptor.update(msg) #+  self.encryptor.finalize()
        return ct
    
csprng = CSPRNG_CRT()
print(csprng.rand())
print(csprng.rand())

b'f\xe9K\xd4\xef\x8a,;\x88L\xfaY\xca4+.'
b'X\xe2\xfc\xce\xfa~0a6\x7f\x1dW\xa4\xe7EZ'


# TRNG

 ## Entropy
 - The function **time_ns** produces certain amount of entropy per call.  
 - Implement function `entropy` that will produce required bits (`req_bits`) of entropy per call.
 - What is the appropriate way how to combine more values together? 
  - Can we use XOR (bitwise operator: ^) to combine random values? 
  - Or we should use hash function? 
  - What are pros and cons of XOR and hash?

In [86]:
def time_ns(state_size=8):
    return time.time_ns().to_bytes(state_size, byteorder='big') 

def XOR(bytes1, bytes2):
    return bytes(a ^ b for (a, b) in zip(bytes1, bytes2))

def entropy(req_bits):
    pass

## ANSIX931
  1. Implement ANSIX931
  2. Use appropriate source of entropy (os.urandom(), secrets.token_bytes()) to reseed the generator.

In [87]:
def AES_ECB(msg, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB())
    encryptor = cipher.encryptor() 
    ct = encryptor.update(msg) +  encryptor.finalize()
    return ct

import time
class ANSIX931: 
    def __init__(self, state_size=16, cipher=AES_ECB, key=b"\x00"*16) :
        self.state_size = state_size
        self.state = b"\x00"*state_size
        self.key = key
        
    def seed(self, seed):
        self.state = seed
        
    def random(self): 
        T = time.time_ns().to_bytes(self.state_size, byteorder='big') 
        tmp1 = AES_ECB(T, self.key) 
        tmp2 = XOR(tmp1, self.state)
        R = AES_ECB(tmp2, self.key) 
        self.state = XOR(tmp1, R)
        return R

RNG = ANSIX931()
RNG.random().hex()

'1dec123a576af52c1f9da11a32fd0f5d'